In [ ]:
from datetime import datetime

from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.dates as mpld
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import minmax_scale

filepath = "~/EDI_AGG_OUTPATIENT/"
DTYPES = {"servicedate": str, "patCountyFIPS": str,
            "Denominator": int, "Flu1": int,
            "Covid_like": int, "Flu_like": int,
            "Mixed": int, "patAgeGroup": str,
            "patHRRname": str, "patHRRid": float}
date = "servicedate"

In [ ]:
df = pd.read_csv(filepath + "EDI_AGG_OUTPATIENT_20200823_0250CDT.csv.gz", dtype = DTYPES,
           parse_dates = ["servicedate"])

In [ ]:
df.shape

In [ ]:
df.drop(columns=["patHRRname", "patHRRid", "patAgeGroup"], inplace=True)
df.dropna(inplace=True)
df = df[((df[date] <= datetime(2020, 8, 22)) & (df[date] > datetime(2020, 3, 1)))]
df = df.groupby([date, "patCountyFIPS"]).sum().reset_index()
df.shape

In [ ]:
## -- check out specific FIPS
# foo = df[df["patCountyFIPS"] == "20029"]
# foo = foo.sort_values(by=['servicedate'])
# plt.plot(foo.servicedate, foo.Flu_like.rolling(7).sum())
# #plt.axhline(y=100)
# plt.xticks(rotation=90)
# plt.show()
# foo.servicedate

In [ ]:
# -- map counties to state level 
state_map = pd.read_csv("~/Delphi/covid-19/geographical_scope/02_20_uszips.csv",
                        usecols=["fips", "state_id"],
                        dtype={"state_id": str},
                        converters={"fips": lambda x:  str(x).zfill(5)})
locs = state_map["state_id"].unique()
state_map.drop_duplicates(inplace=True)
data = df.merge(state_map, how="left", left_on="patCountyFIPS", right_on="fips")
data.dropna(inplace=True)
data.drop(columns=["patCountyFIPS", "fips"], inplace=True)
data = data.groupby(["state_id", date]).sum()

In [ ]:
# -- start pdf document
pdf_pages = PdfPages("flu1.pdf")
n_plot = len(locs)
n_plots_per_page = 25

# -- init plotting axis and counter
fig, axs = None, None
j = 0

for i, loc in enumerate(locs):
    # start new page if needed
    if i % n_plots_per_page == 0:
        fig, axs = plt.subplots(5, 5, figsize=(10, 10))
        axs = axs.ravel()
        j = 0

    # plot
    x = data.loc[loc]

    #axs[j].plot(x.index, x.Flu_like, alpha=0.2)
    #x_smooth = minmax_scale(x["Covid_like"].rolling(7).mean() + x["Flu_like"].rolling(7).mean() +
    #                        x["Mixed"].rolling(7).mean() - x["Flu1"].rolling(7).mean())
    #axs[j].plot(x.index, x_smooth, label="numerator")
    #x_smooth_d = minmax_scale(x["Denominator"].rolling(7).mean())
    #axs[j].plot(x.index, x_smooth_d, label="denominator")
    #ratio = x_smooth / (x_smooth_d)
    #ratio[np.isinf(ratio)] = 0
    #axs[j].plot(x.index, minmax_scale(ratio), label="ratio")
    axs[j].plot(x.index, 100 * x.Flu1.rolling(7).mean() / x["Denominator"].rolling(7).mean(), 
                label="100*Flu_like/Denominator")
    axs[j].legend(fontsize="xx-small")
    axs[j].set_title(loc)
    axs[j].xaxis.set_major_formatter(mpld.DateFormatter('%m-%d'))
    axs[j].tick_params(axis='both', which='major', labelsize=5, labelrotation=90)
    #axs[j].set_yticks([]) # obscure y-axis

    # close the page if needed
    if (i + 1) % n_plots_per_page == 0 or (i + 1) == n_plot:
        plt.tight_layout()
        pdf_pages.savefig(fig)
        plt.close()
    j += 1

pdf_pages.close()